In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
from datetime import datetime

In [4]:
df = pd.read_csv('revs_scored_jan.tsv', sep='\t', header=0)

In [5]:
days = df.rev_timestamp.map(lambda ts: datetime.utcfromtimestamp(ts).day)
df['day'] = days

In [6]:
dfbots = df[df.day<22]
dfbots = dfbots[dfbots.user_is_bot]

In [7]:
dfrevertedbots = dfbots[dfbots.is_reverted]
dfrevertedbots = dfrevertedbots[dfrevertedbots.seconds_to_revert.astype('str').astype('int')<86400]
dfrevertedbots.revert_id = dfrevertedbots.revert_id.astype('int')

In [8]:
dfreverters = df[df.is_revert]

In [9]:
# merging two datafraemes to create a table of nonhuman and human reverters that reverted a bot

dfrevertedbots = pd.merge(dfrevertedbots, dfreverters, 
             how='inner', 
             left_on='revert_id', 
             right_on='rev_id', 
             suffixes=('', '_reverter')
)


In [10]:
dfrevertedbots.columns

Index(['rev_timestamp', 'page_id', 'rev_id', 'prev_rev_id', 'is_minor',
       'user_text', 'user_id', 'seconds_to_prev', 'curr_bytes', 'delta_bytes',
       'has_edit_summary', 'is_reverted', 'is_revert',
       'is_reverted_to_by_other', 'is_self_reverted', 'is_self_revert',
       'revert_target_id', 'revert_set_size', 'revert_id', 'seconds_to_revert',
       'damaging_pred', 'goodfaith_pred', 'model_version', 'user_is_bot',
       'user_is_trusted', 'user_edit_count', 'page_rev_count',
       'page_namespace', 'is_page_redirect', 'day', 'rev_timestamp_reverter',
       'page_id_reverter', 'rev_id_reverter', 'prev_rev_id_reverter',
       'is_minor_reverter', 'user_text_reverter', 'user_id_reverter',
       'seconds_to_prev_reverter', 'curr_bytes_reverter',
       'delta_bytes_reverter', 'has_edit_summary_reverter',
       'is_reverted_reverter', 'is_revert_reverter',
       'is_reverted_to_by_other_reverter', 'is_self_reverted_reverter',
       'is_self_revert_reverter', 'revert_ta

In [20]:
dfrevertedbots = dfrevertedbots[['rev_id','user_text','revert_id','user_text_reverter', 'is_self_revert_reverter','user_is_bot_reverter', 'page_namespace','day']]

In [21]:
dfrevertedbots.rename(columns={'is_self_revert_reverter': 'self_reverted', 'user_is_bot_reverter': 'reverter_is_bot'})

,rev_id,user_text,revert_id,user_text_reverter,self_reverted,reverter_is_bot,page_namespace,day
0,876227358,Cydebot,876373440,Cydebot,True,True,2,1
1,876264984,Cydebot,876373440,Cydebot,True,True,2,1
2,876274215,Cydebot,876373440,Cydebot,True,True,2,1
3,876274593,Cydebot,876373440,Cydebot,True,True,2,1
4,876275010,Cydebot,876373440,Cydebot,True,True,2,1
...,...,...,...,...,...,...,...,...
8480,879554462,SineBot,879554618,Larry Hockett,False,False,1,21
8481,879554569,Amalthea (bot),879555726,Amalthea (bot),True,True,2,21
8482,879555445,DatBot,879558700,HBC AIV helperbot5,False,True,4,21
8483,879555579,Lowercase sigmabot II,879577619,7&6=thirteen,False,False,118,21


In [17]:
dfrevertedbotsn